In [1]:
import torch
from torchinfo import summary

import model_mobilenetv2_mini_Resnet_Brevitas as brevitas_model
from brevitas.export import export_qonnx

# Mobilenet Resnet Model

In [2]:
model_qnn = brevitas_model.MobileNetV2_MINI_RESNET().to('cpu')

/home/gmoreno/uav/finn/deps/brevitas/src/brevitas/nn/mixin/base.py:77: UserWarning: Keyword arguments are being passed but they not being used.
  warn('Keyword arguments are being passed but they not being used.')


In [3]:
input_shape = (1, 3, 224, 224)
# print(summary(model_qnn, input_size=input_shape))

In [4]:
models_folder = './step_by_step'
model_qnn_filename = models_folder + '/MobilenetV2_Resnet__QONNX.onnx' 

In [5]:
model_qnn.eval();
export_qonnx(model_qnn, torch.randn(input_shape), model_qnn_filename);

# FINN Flow

## Load Model and View

In [6]:
from finn.util.visualization import showSrc, showInNetron
from qonnx.util.cleanup import cleanup as qonnx_cleanup

In [7]:
showInNetron(model_qnn_filename)

Serving './step_by_step/MobilenetV2_Resnet__QONNX.onnx' at http://0.0.0.0:8083


## Clean

In [8]:
qonnx_clean_filename = models_folder + '/01_clean.onnx'
qonnx_cleanup(model_qnn_filename, out_file=qonnx_clean_filename)

In [9]:
showInNetron(qonnx_clean_filename)

Stopping http://0.0.0.0:8083
Serving './step_by_step/01_clean.onnx' at http://0.0.0.0:8083


## Convert to FINN

In [10]:
from qonnx.core.modelwrapper import ModelWrapper

In [11]:
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

In [12]:
model = ModelWrapper(qonnx_clean_filename)
model = model.transform(ConvertQONNXtoFINN())
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/gemm_to_matmul.py:57: UserWarning: The GemmToMatMul transformation only offers explicit support for version 9 of the Gemm node, but the ONNX version of the supplied model is 14. Thus the transformation may fail or return incomplete results.
  warnings.warn(


In [13]:
finn_tidy = models_folder + '/02_finn_tidy.onnx'
model.save(finn_tidy)

In [14]:
showInNetron(finn_tidy)

Stopping http://0.0.0.0:8083
Serving './step_by_step/02_finn_tidy.onnx' at http://0.0.0.0:8083


## Preprocess

In [15]:
from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType
from qonnx.transformation.infer_datatypes import InferDataTypes

In [16]:
model = ModelWrapper(finn_tidy)
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = models_folder + "/prepro_node.onnx"
export_qonnx(totensor_pyt, torch.randn(ishape), chkpt_preproc_name)
qonnx_cleanup(chkpt_preproc_name, out_file=chkpt_preproc_name)
pre_model = ModelWrapper(chkpt_preproc_name)
pre_model = pre_model.transform(ConvertQONNXtoFINN())

# join preprocessing and core model
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


### Tidy again

In [17]:
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

In [18]:
finn_prepro = models_folder + '/03_finn_prepro.onnx'
model.save(finn_prepro)

In [19]:
showInNetron(finn_prepro)

Stopping http://0.0.0.0:8083
Serving './step_by_step/03_finn_prepro.onnx' at http://0.0.0.0:8083


## Streamline

In [20]:
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul

from qonnx.transformation.change_datalayout import ChangeDataLayoutQuantAvgPool2d
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors

from finn.transformation.streamline import Streamline
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC
from finn.transformation.streamline.reorder import MoveLinearPastEltwiseAdd

from finn.transformation.streamline.reorder import MoveMulPastFork

In [21]:
model = ModelWrapper(finn_prepro)
# model = model.transform(absorb.AbsorbMulIntoMultiThreshold())
# model = model.transform(absorb.AbsorbAddIntoMultiThreshold())

model = model.transform(MoveMulPastFork())
model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

In [22]:
finn_mul_past_fork = models_folder + '/040_finn_mul_past_fork.onnx'
model.save(finn_mul_past_fork)

In [23]:
showInNetron(finn_mul_past_fork)

Stopping http://0.0.0.0:8083
Serving './step_by_step/040_finn_mul_past_fork.onnx' at http://0.0.0.0:8083


In [24]:
model = ModelWrapper(finn_mul_past_fork)

In [25]:
model = model.transform(MoveLinearPastEltwiseAdd())
model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

/home/gmoreno/uav/finn/src/finn/transformation/streamline/absorb.py:66: UserWarning: Threshold or add bias not constant, skipping
  warnings.warn("Threshold or add bias not constant, skipping")


In [26]:
finn_move_mul_past_add = models_folder + '/041_finn_move_mul_past_add.onnx'
model.save(finn_move_mul_past_add)

In [27]:
showInNetron(finn_move_mul_past_add)

Stopping http://0.0.0.0:8083
Serving './step_by_step/041_finn_move_mul_past_add.onnx' at http://0.0.0.0:8083


In [28]:
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from finn.transformation.streamline.reorder import MoveTransposePastFork 

In [29]:
model = ModelWrapper(finn_move_mul_past_add)

In [30]:
model = model.transform(to_hw.InferAddStreamsLayer())
# model = model.transform(MoveTransposePastFork())
model = model.transform(LowerConvsToMatMul())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(absorb.AbsorbConsecutiveTransposes())


In [31]:
finn_add_to_hw = models_folder + '/042_finn_add_to_hw.onnx'
model.save(finn_add_to_hw)

In [32]:
showInNetron(finn_add_to_hw)

Stopping http://0.0.0.0:8083
Serving './step_by_step/042_finn_add_to_hw.onnx' at http://0.0.0.0:8083


In [33]:
model = ModelWrapper(finn_add_to_hw)

In [34]:
model = model.transform(ChangeDataLayoutQuantAvgPool2d())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

In [35]:
finn_avgpool = models_folder + '/043_finn_avgpool.onnx'
model.save(finn_avgpool)

In [36]:
showInNetron(finn_avgpool)

Stopping http://0.0.0.0:8083
Serving './step_by_step/043_finn_avgpool.onnx' at http://0.0.0.0:8083


# Old Streamline plus some test: it does not work

In [37]:
# model = ModelWrapper(finn_prepro)
# # model = model.transform(MoveLinearPastEltwiseAdd())
# model = model.transform(absorb.AbsorbAddIntoMultiThreshold())
# model = model.transform(absorb.AbsorbMulIntoMultiThreshold())

# model = model.transform(MoveScalarLinearPastInvariants())
# model = model.transform(Streamline())
# model = model.transform(LowerConvsToMatMul())
# model = model.transform(MakeMaxPoolNHWC())
# model = model.transform(ChangeDataLayoutQuantAvgPool2d())
# model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())

# model = model.transform(Streamline())
# model = model.transform(InferDataLayouts())
# model = model.transform(RemoveUnusedTensors())

In [38]:
# finn_streamline = models_folder + '/04_finn_streamline.onnx'
# model.save(finn_streamline)

In [39]:
# showInNetron(finn_streamline)

# To HW Layers

In [40]:
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten

from qonnx.custom_op.registry import getCustomOp

In [41]:
model = ModelWrapper(finn_avgpool)

In [42]:
model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())

model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

In [43]:
finn_hw_mvau = models_folder + '/044_finn_hw_mvau.onnx'
model.save(finn_hw_mvau)

In [44]:
showInNetron(finn_hw_mvau)

Stopping http://0.0.0.0:8083
Serving './step_by_step/044_finn_hw_mvau.onnx' at http://0.0.0.0:8083


In [45]:
model = ModelWrapper(finn_hw_mvau)

In [46]:
model = model.transform(to_hw.InferConvInpGen())
model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

In [47]:
finn_hw_convs = models_folder + '/045_finn_hw_convs.onnx'
model.save(finn_hw_convs)

In [48]:
showInNetron(finn_hw_convs)

Stopping http://0.0.0.0:8083
Serving './step_by_step/045_finn_hw_convs.onnx' at http://0.0.0.0:8083


In [49]:
model = ModelWrapper(finn_hw_convs)

In [50]:
model = model.transform(to_hw.InferPool())
model = model.transform(to_hw.InferConvInpGen())

model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

In [51]:
finn_hw_pool = models_folder + '/046_finn_hw_pool.onnx'
model.save(finn_hw_pool)

In [52]:
showInNetron(finn_hw_pool)

Stopping http://0.0.0.0:8083
Serving './step_by_step/046_finn_hw_pool.onnx' at http://0.0.0.0:8083


In [53]:
model = ModelWrapper(finn_hw_pool)

In [54]:
model = model.transform(to_hw.InferThresholdingLayer())
model = model.transform(RemoveCNVtoFCFlatten())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

In [55]:
finn_hw_multithres_flaten = models_folder + '/047_finn_hw_multithres_flaten.onnx'
model.save(finn_hw_multithres_flaten)

In [56]:
showInNetron(finn_hw_multithres_flaten)

Stopping http://0.0.0.0:8083
Serving './step_by_step/047_finn_hw_multithres_flaten.onnx' at http://0.0.0.0:8083


In [57]:
model = ModelWrapper(finn_hw_multithres_flaten)
model = model.transform(to_hw.InferVectorVectorActivation())
model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

In [58]:
finn_hw_dw_convs = models_folder + '/047_finn_hw_dw_convs.onnx'
model.save(finn_hw_dw_convs)

In [59]:
showInNetron(finn_hw_dw_convs)

Stopping http://0.0.0.0:8083
Serving './step_by_step/047_finn_hw_dw_convs.onnx' at http://0.0.0.0:8083


In [60]:
model = ModelWrapper(finn_hw_dw_convs)
parent_model = model.transform(CreateDataflowPartition())

In [61]:
finn_parent_filename = models_folder + '/00_finn_dataflow_parent.onnx'
parent_model.save(finn_parent_filename)

In [62]:
showInNetron(finn_parent_filename)

Stopping http://0.0.0.0:8083
Serving './step_by_step/00_finn_dataflow_parent.onnx' at http://0.0.0.0:8083


In [63]:
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_filename = sdp_node.get_nodeattr("model")
dataflow_model = ModelWrapper(dataflow_filename)

In [64]:
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers

In [65]:
from finn.util.basic import pynq_part_map
# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z1"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

In [66]:
# save the dataflow partition with a different name for easier access
# and specialize the layers to HLS variants
dataflow_model = dataflow_model.transform(SpecializeLayers(fpga_part))

dataflow_model = dataflow_model.transform(GiveUniqueNodeNames())
dataflow_model = dataflow_model.transform(GiveReadableTensorNames())

finn_dataflow_filename = models_folder + '/20_finn_dataflow_model.onnx'
dataflow_model.save(finn_dataflow_filename)

In [67]:
showInNetron(finn_dataflow_filename)

Stopping http://0.0.0.0:8083
Serving './step_by_step/20_finn_dataflow_model.onnx' at http://0.0.0.0:8083
